# Importando Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Baixar recursos adicionais do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Carregando DataFrame

In [ ]:
#https://www.kaggle.com/datasets/luisfredgs/imdb-ptbr?resource=download
df = pd.read_csv('/content/imdb-reviews-pt-br.csv')
df

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


# Análise Exploratória dos Dados

In [ ]:
# Verificando o cabeçalho do dataset (primeiras 5 linhas)
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
# Verificando o rodapé do dataset (últimas 5 linhas)
df.tail()

,id,text_en,text_pt,sentiment
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos
49458,49460,Working-class romantic drama from director Mar...,O drama romântico da classe trabalhadora do di...,pos


In [ ]:
# Verificando o formato do dataset (número de linhas e colunas)
df.shape

(49459, 4)

In [ ]:
# Verificando o tipo das features
df.dtypes

id            int64
text_en      object
text_pt      object
sentiment    object
dtype: object

In [ ]:
# Verificando se existem valores nulos
df.isnull().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [ ]:
# Verificando se existem linhas duplicadas
df.duplicated().sum()

0

In [ ]:
# Dropar a coluna 'text_en', pois contém as mesmas informações da coluna 'text_pt', porém em outro idioma
df = df.drop('text_en', axis=1)

In [ ]:
# Verificando a porcentagem (%) dos valores da coluna 'Sentimento'
df['sentiment'].value_counts()/df.shape[0]

neg    0.500718
pos    0.499282
Name: sentiment, dtype: float64

In [ ]:
# Contando linhas por sentimento para ver o mapeamento
df.groupby(['sentiment']).count()

,id,text_pt
sentiment,,
neg,24765,24765
pos,24694,24694


In [ ]:
# Removendo pontuações
df['text_pt'] = df['text_pt'].str.translate(str.maketrans("", "", string.punctuation))

In [ ]:
# Realizando tokenização
df['tokens'] = df['text_pt'].apply(word_tokenize)
df['tokens']

0        [Mais, uma, vez, o, Sr, Costner, arrumou, um, ...
1        [Este, é, um, exemplo, do, motivo, pelo, qual,...
2        [Primeiro, de, tudo, eu, odeio, esses, raps, i...
3        [Nem, mesmo, os, Beatles, puderam, escrever, m...
4        [Filmes, de, fotos, de, latão, não, é, uma, pa...
                               ...                        
49454    [Como, a, média, de, votos, era, muito, baixa,...
49455    [O, enredo, teve, algumas, reviravoltas, infel...
49456    [Estou, espantado, com, a, forma, como, este, ...
49457    [A, Christmas, Together, realmente, veio, ante...
49458    [O, drama, romântico, da, classe, trabalhadora...
Name: tokens, Length: 49459, dtype: object

In [ ]:
# Removendo stopwords - palavras que podem ser consideradas irrelevantes
stop_words = set(stopwords.words('portuguese'))
df['tokens'] = df['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])

In [ ]:
# Realizando normalização de texto
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

In [ ]:
# Realizando conversão para minúsculas
df['tokens'] = df['tokens'].apply(lambda tokens: [token.lower() for token in tokens])

In [ ]:
# Resultado do pré-processamento
print(df['tokens'])

0        [vez, sr, costner, arrumou, filme, tempo, nece...
1        [exemplo, motivo, maioria, filmes, ação, mesmo...
2        [primeiro, tudo, odeio, rap, imbecis, poderiam...
3        [beatles, puderam, escrever, músicas, todos, g...
4        [filmes, fotos, latão, palavra, apropriada, ve...
                               ...                        
49454    [média, votos, baixa, fato, funcionário, locad...
49455    [enredo, algumas, reviravoltas, infelizes, ina...
49456    [espantado, forma, filme, maioria, outros, méd...
49457    [christmas, together, realmente, veio, ante, t...
49458    [drama, romântico, classe, trabalhadora, diret...
Name: tokens, Length: 49459, dtype: object


# Treinando o Modelo

In [ ]:
# Extraindo recursos
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tokens'].apply(' '.join))  # Convertendo a lista de tokens em uma string
y = df['sentiment']

In [ ]:
# Dividindo dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Treinando o modelo em SVM com os dados de treino
svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [ ]:
# Fazendo previsões nos dados de teste
y_pred = svm.predict(X_test)

# Avaliando o Modelo

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[4459,  579],
       [ 465, 4389]])

In [ ]:
# Calculando a Acurácia do Modelo
accuracy = accuracy_score(y_test, y_pred)
print('Acurácia: ',accuracy)

Acurácia:  0.8944601698342095


In [ ]:
# Calculando a Precisão do Modelo
precision = precision_score(y_test, y_pred, average='macro')
print('Precisão: ',precision)

Precisão:  0.8945093439605678


In [ ]:
# Calculando o Recall do Modelo
recall = recall_score(y_test, y_pred, average='macro')
print('Recall: ',recall)

Recall:  0.8946380806243379


In [ ]:
# Calculando o F1-Score do Modelo
f1 = f1_score(y_test, y_pred, average='macro')
print('F1-Score: ',f1)

F1-Score:  0.8944548845785028


# Conclusão

### Em resumo, após um período de aproximandamente 65min para o processamento do treinamento ser concluído, o modelo de análise de sentimentos em questão parece ter um desempenho satisfatório, com uma alta acurácia e F1-score, além de indicar um bom equilíbrio entre a precisão e o recall, o que significa que ele é capaz de classificar corretamente os sentimentos expressos nos textos em grande parte dos casos. 